<a href="https://colab.research.google.com/github/Rhqo/LLM_Study/blob/main/chunking_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Max token window chunking

## Get PDF File

In [57]:
!pip install PyPDF2

In [77]:
import PyPDF2
from tqdm import tqdm
import re
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
with open ('/content/drive/MyDrive/Foundations.Of.Algorithms.5th.Edition.pdf', 'rb') as file:
  reader = PyPDF2.PdfReader(file, strict=False)

  principles_of_ds = ''

  for page in tqdm(reader.pages):
    text = page.extract_text()
    principles_of_ds += '\n\n' + text[text.find(']')+2:]

principles_of_ds = principles_of_ds.strip()

100%|██████████| 695/695 [02:47<00:00,  4.14it/s]


## Overlapping chunks

In [55]:
!pip install tiktoken

In [80]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

In [81]:
def overlapping_chunks(text, max_tokens=500, overlapping_factor = 5):
  sentences = re.split(r'[.?!]', text)
  n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]

  chunks, tokens_so_far, chunk = [], 0, []

  for sentence, token in zip(sentences, n_tokens):
    if tokens_so_far + token > max_tokens:
      chunk.append(". ".join(chunk)+".")

      if overlapping_factor > 0:
        chunk = chunk[-overlapping_factor:]
        tokens_so_far = sum([lne(tokenizer.encode(c)) for c in chunk])
      else:
        chunk = []
        tokens_so_far = 0

      if token > max_tokens:
        continue
      chunk.append(sentence)
      tokens_so_far += token + 1
  return chunks

In [84]:
split = overlapping_chunks(principles_of_ds, overlapping_factor=0)
print([len(tokenizer.encode(t)) for t in split])
print(len(split))
#avg_length = sum([len(tokenizer.encode(t)) for t in split]) / len(split)

[]
0
